## Base Inference

In [1]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
from PIL import Image

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-infographics-vqa-large").to("cuda")
processor = Pix2StructProcessor.from_pretrained("google/pix2struct-infographics-vqa-large")


In [2]:
image = Image.open("/home/jjh/level3-cv-productserving-cv-10/data/images/10065.jpeg")
question = "Which market crash had the lowest impact on the S&P 500, Dot-com crash, Coronavirus crash, or Great recession ?"
inputs = processor(images=image, text=question, return_tensors="pt").to("cuda")

In [3]:
ins = processor(images = image,text=question,return_tensors='pt').to('cuda')

In [4]:
predictions = model.generate(**inputs)
pred = processor.decode(predictions[0], skip_special_tokens=True)
print(pred)

/home/jjh/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Coronavirus crash


## train

In [5]:
import os
import torch
from torch.utils.data import Dataset
import json
from PIL import Image

In [6]:
from transformers import AutoProcessor
auto_processor = AutoProcessor.from_pretrained("google/pix2struct-infographics-vqa-large")
auto_processor.image_processor.is_vqa = False

In [7]:
file_list = os.listdir("/home/jjh/level3-cv-productserving-cv-10/data/images/")
with open("../data/qas/infographicsVQA_train_v1.0.json") as f:
    json_data = json.load(f)
image_name = []
q = []
for d in json_data["data"]:
    image_name.append(d["image_local_name"])
    q.append(d["question"])

In [35]:
class Pix2StructDataset(Dataset):
    def __init__(self, image_dir, json_dir, processor, train):
        self.img_dir = image_dir
        with open(json_dir) as f:
            self.json_data = json.load(f)
        self.processor = processor
        self.file_list = os.listdir(image_dir)
        self.train = train
        
    def __getitem__(self, index): 
        # image_name = self.file_list[index]
        # img = Image.open(os.path.join(self.img_dir, image_name))

        # for d in self.json_data['data']:
        #     if d["image_local_name"] == image_name:
        #         q = d["question"]
        #         if self.train:
        #             a = d["answers"]
        
        # inputs = self.processor(images=img, text=q, return_tensors="pt").to('cuda')

        # if self.train:
        #     label = auto_processor(text=a, return_tensors="pt").input_ids
        #     inputs["labels"] = label
        data = self.json_data["data"][index]
        image_name = data["image_local_name"]
        img = Image.open(os.path.join(self.img_dir, image_name))
        q = data["question"]
        inputs = self.processor(images=img, text=q, return_tensors="pt").to('cuda')
        if self.train:
            a = data["answers"]
            # label = auto_processor(text=a, return_tensors="pt").input_ids
            label = auto_processor(text=a, return_tensors="pt", padding=True, truncation=True).input_ids
            inputs["labels"] = label
        return inputs
                    
    def __len__(self): 
        return len(self.file_list)

In [36]:
img_dir = '/home/jjh/level3-cv-productserving-cv-10/data/images/'
train_dataset = Pix2StructDataset(image_dir=img_dir, json_dir='../data/qas/infographicsVQA_train_v1.0.json', processor=processor, train=True)
val_dataset = Pix2StructDataset(image_dir=img_dir, json_dir='../data/qas/infographicsVQA_val_v1.0_withQT.json', processor=processor, train=True)
test_dataset = Pix2StructDataset(image_dir=img_dir, json_dir='../data/qas/infographicsVQA_test_v1.0.json', processor=processor, train=False)


In [37]:
# index = 3
# data = json_data["data"][index]
# image_name = data["image_local_name"]
# img = Image.open(os.path.join(img_dir, image_name))
# q = data["question"]
# inputs = processor(images=img, text=q, return_tensors="pt").to('cuda')
# if  True:
#     a = data["answers"]
#     label = auto_processor(text=a, return_tensors="pt", padding=True, truncation=True).input_ids
#     inputs["labels"] = label
# print(inputs)

In [38]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1)

In [41]:
for i in range(len(train_dataset)):
    train_dataset[i]

KeyboardInterrupt: 

In [40]:
from PIL import Image
import requests

from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for epoch in range(10):
    for idx, batch in enumerate(dataloader):
        f = batch["flattened_patches"]
        a = batch["attention_mask"]
        labels = batch["labels"]
        outputs = model(flattened_patches = f, attention_mask = a, labels=labels)
        loss = outputs.loss
        print(f"{loss.item():.5f}")

/home/jjh/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2046x770 and 768x1536)